# Diffe Hellman Implementation

In [1]:
from random import randint

if __name__ == '__main__':

	# Both the persons will be agreed upon the
	# public keys G and P
	# A prime number P is taken
	P = 13931
	
	# A primitive root for P, G is taken
	G = 123
	
	
	print('The Value of P is :%d'%(P))
	print('The Value of G is :%d'%(G))
	
	# Alice will choose the private key a
	a = 4
	print('The Private Key a for Alice is :%d'%(a))
	
	# gets the generated key
	x = int(pow(G,a,P))
	
	# Bob will choose the private key b
	b = 3
	print('The Private Key b for Bob is :%d'%(b))
	
	# gets the generated key
	y = int(pow(G,b,P))
	
	
	# Secret key for Alice
	ka = int(pow(y,a,P))
	
	# Secret key for Bob
	kb = int(pow(x,b,P))
	
	print('Secret key for the Alice is : %d'%(ka))
	print('Secret Key for the Bob is : %d'%(kb))

The Value of P is :13931
The Value of G is :123
The Private Key a for Alice is :4
The Private Key b for Bob is :3
Secret key for the Alice is : 3202
Secret Key for the Bob is : 3202


# MiTM Attack
Man in the Middle Attack on Diffe Hellman

In [2]:
import random

# public keys are taken
# p is a prime number
# g is a primitive root of p
p = 13931
g = 123


class A:
	def __init__(self):
		# Generating a random private number selected by alice
		self.n = random.randint(1, p)	

	def publish(self):
		# generating public values
		return (g**self.n)%p

	def compute_secret(self, gb):
		# computing secret key
		return (gb**self.n)%p


class B:
	def __init__(self):
		# Generating a random private number selected for alice
		self.a = random.randint(1, p)
		# Generating a random private number selected for bob
		self.b = random.randint(1, p)
		self.arr = [self.a,self.b]

	def publish(self, i):
		# generating public values
		return (g**self.arr[i])%p

	def compute_secret(self, ga, i):
		# computing secret key
		return (ga**self.arr[i])%p


alice = A()
bob = A()
eve = B()

# Printing out the private selected number by Alice and Bob
print(f'Alice selected (a) : {alice.n}')
print(f'Bob selected (b) : {bob.n}')
print(f'Eve selectd private number for Alice (c) : {eve.a}')
print(f'Eve selectd private number for Bob (d) : {eve.b}')

# Generating public values
ga = alice.publish()
gb = bob.publish()
gea = eve.publish(0)
geb = eve.publish(1)
print(f'Alice published (ga): {ga}')
print(f'Bob published (gb): {gb}')
print(f'Eve published value for Alice (gc): {gea}')
print(f'Eve published value for Bob (gd): {geb}')

# Computing the secret key
sa = alice.compute_secret(gea)
sea = eve.compute_secret(ga,0)
sb = bob.compute_secret(geb)
seb = eve.compute_secret(gb,1)
print(f'Alice computed (S1) : {sa}')
print(f'Eve computed key for Alice (S1) : {sea}')
print(f'Bob computed (S2) : {sb}')
print(f'Eve computed key for Bob (S2) : {seb}')

Alice selected (a) : 8952
Bob selected (b) : 7399
Eve selectd private number for Alice (c) : 13042
Eve selectd private number for Bob (d) : 3152
Alice published (ga): 4604
Bob published (gb): 12427
Eve published value for Alice (gc): 1783
Eve published value for Bob (gd): 2162
Alice computed (S1) : 45
Eve computed key for Alice (S1) : 45
Bob computed (S2) : 13648
Eve computed key for Bob (S2) : 13648


# ECDSA
Elliptic Curve Digital Signature Algo

In [3]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.exceptions import InvalidSignature


private_value = 0x63bd3b01c5ce749d87f5f7481232a93540acdb0f7b5c014ecd9cd32b041d6f33
curve = ec.SECP256R1()
signature_algorithm = ec.ECDSA(hashes.SHA256())

# Make private and public keys from the private value + curve
priv_key = ec.derive_private_key(private_value, curve, default_backend())
pub_key = priv_key.public_key()
print('Private key: 0x%x' % priv_key.private_numbers().private_value)
print('Public point (Uncompressed): 0x%s' % pub_key.public_bytes(serialization.Encoding.X962, serialization.PublicFormat.UncompressedPoint).hex())

# Sign some data
data = b"this is some data to sign"
signature = priv_key.sign(data, signature_algorithm)
print('Signature: 0x%s' % signature.hex())

# Verify
try:
    pub_key.verify(signature, data, signature_algorithm)
    print('Verification OK')
except InvalidSignature:
    print('Verification failed')

Private key: 0x63bd3b01c5ce749d87f5f7481232a93540acdb0f7b5c014ecd9cd32b041d6f33
Public point (Uncompressed): 0x04017655e42a892cc71bccedcb1cd421d03530e1d7edb52cef143c5562c4c6f0129fa5a37738013e64a1ff0e6cb7068815a13000eb162cb7a0214dfcf3c8fa101c
Signature: 0x30450221008879dff1cd6022b7bc36851aa8f1d5dfbfcf490e4adbb3eff093a10c69f5d73702206a105159024b224992df049a79b01dccd9b2e754ffcfda2f63c84ef713d0a5b1
Verification OK


# Integration of ECDSA and ECC Diffe Hellman

In [4]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.exceptions import InvalidSignature
from random import randint


# Alice will choose the private key a
private_value_a = 0x63bd3b01c5ce749d87f5f7481232a93540acdb0f7b5c014ecd9cd32b041d6f33
    
# Bob will choose the private key b
private_value_b = 0xef235aacf90d9f4aadd8c92e4b2562e1d9eb97f0df9ba3b508258739cb013db2
    
# Elliptic Curve
curve = ec.SECP256R1()
    
# Algoritim for checking digital signature
signature_algorithm = ec.ECDSA(hashes.SHA256())
    
    
# Make private and public keys from the private value + curve
priv_key_a = ec.derive_private_key(private_value_a, curve, default_backend())
pub_key_a = priv_key_a.public_key()
print('Private key of Alice: 0x%x' % priv_key_a.private_numbers().private_value)
print('Public point of Alice (Uncompressed): 0x%s' % pub_key_a.public_bytes(serialization.Encoding.X962, serialization.PublicFormat.UncompressedPoint).hex())


# Make private and public keys from the private value + curve
priv_key_b = ec.derive_private_key(private_value_b, curve, default_backend())
pub_key_b = priv_key_b.public_key()
print('Private key of Bob: 0x%x' % priv_key_b.private_numbers().private_value)
print('Public point of Bob (Uncompressed): 0x%s' % pub_key_b.public_bytes(serialization.Encoding.X962, serialization.PublicFormat.UncompressedPoint).hex())

    
# Sign some data
data = b"I am Alice"
signature = priv_key_a.sign(data, signature_algorithm)
print('Signature: 0x%s' % signature.hex())
    
# Verify
try:
    pub_key_a.verify(signature, data, signature_algorithm)
    print('Verification OK')
except InvalidSignature:
    print('Verification failed')
        
# Verify
try:
    pub_key_b.verify(signature, data, signature_algorithm)
    print('Verification OK')
except InvalidSignature:
    print('Verification failed')

Private key of Alice: 0x63bd3b01c5ce749d87f5f7481232a93540acdb0f7b5c014ecd9cd32b041d6f33
Public point of Alice (Uncompressed): 0x04017655e42a892cc71bccedcb1cd421d03530e1d7edb52cef143c5562c4c6f0129fa5a37738013e64a1ff0e6cb7068815a13000eb162cb7a0214dfcf3c8fa101c
Private key of Bob: 0xef235aacf90d9f4aadd8c92e4b2562e1d9eb97f0df9ba3b508258739cb013db2
Public point of Bob (Uncompressed): 0x0482f94f8cc1592cfd3df5ae015a15b8bd70d35716df42bd6f10edcf251c7b55250c44b239c35c60a0ba2cb6dc70e059a4ec869bb00390abf0de1fdb0c4c7dc00b
Signature: 0x3046022100cd54356973c395e203d62493a37c650d0fd26fb5b94fd41a2558d937709d72af022100c56313e6099503f0f870bd50a9c704aa867f4492702790b995b575457e59dd30
Verification OK
Verification failed
